In [1]:
import logging
from pathlib import Path

from dotenv import dotenv_values

from generator_drainage_units import run_generator_culvert_locations

In [2]:
%load_ext autoreload
%autoreload 2

In [42]:
logging.basicConfig(level=logging.DEBUG)

config = dotenv_values("..\\.env")
base_dir = config["BASE_DIR"]
# case_name = "vallei_en_veluwe"
case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
# case_name = "Leuvenumse_beek"

In [ ]:
case_path = Path(base_dir, case_name)

cg_geer = run_generator_culvert_locations(
    path=case_path,
    distance_vertices=5,
    max_culvert_length=40,
    read_results=False,
    write_results=True,
    create_html_map=True,
)

In [ ]:
case_path = Path(base_dir, case_name)

cg = run_generator_culvert_locations(
    path=case_path,
    distance_vertices=5,
    max_culvert_length=40,
    read_results=False,
    write_results=True,
    create_html_map=True,
)

In [32]:
from shapely.geometry import LineString, Point
import geopandas as gpd

In [35]:
docs_dir = 'C:\\Users\\NL1C2G\\Documents'
culverts = gpd.read_file(Path(docs_dir, "potential_culverts_4.gpkg"))

In [38]:
def generate_outflow_points_overige_watergangen_to_hydroobjecten(gdf):
    culverts_hydro = gdf[
        (gdf["WaterLineType"] == "hydroobjecten")
    ]

    # Extract end points and retain 'unique_id' and 'dangling_code'
    culverts_hydro["end_point"] = culverts_hydro.geometry.apply(
        lambda line: Point(line.coords[-1])
        if isinstance(line, LineString)
        else None
    )

    # Create GeoDataFrame with end points
    end_points_gdf = gpd.GeoDataFrame(
        culverts_hydro[["end_point", "unique_id", "dangling_code"]].dropna(
            subset=["end_point"]
        ),
        geometry="end_point",
        crs=culverts_hydro.crs,
    )

    # Rename the geometry column to 'geometry'
    end_points_gdf = end_points_gdf.rename(
        columns={"end_point": "geometry"}
    ).set_geometry("geometry")

    gdf = end_points_gdf.copy()

    # Save to file
    gdf.to_file(
        Path(docs_dir, "outflow_points_overig_to_hydro.gpkg"),
        layer="outflow_points_overig_to_hydro",
    )

    return gdf

In [ ]:
endpoints = generate_outflow_points_overige_watergangen_to_hydroobjecten(culverts)